In [1]:
i = int(input())

231


In [2]:
print(type(i))

<class 'int'>


In [1]:
import Workbook from openpyxl

SyntaxError: invalid syntax (<ipython-input-1-897b995df71d>, line 1)

In [2]:
from openpyxl import Workbook

In [7]:
import numpy as np

<module 'numpy' from 'S:\\Anaconda\\lib\\site-packages\\numpy\\__init__.py'>

In [3]:
import pandas as pd

a = [1,2,3,4]

In [4]:
s = pd.Series(a)
s

0    1
1    2
2    3
3    4
dtype: int64

In [2]:
def get_title_id():
    '''爬取战疫情首页的每个主题的ID'''
    for page in range(1,21):# 每个页面大约有18个话题
        headers = {
            "User-Agent" : UserAgent().chrome #chrome浏览器随机代理
        }
        time.sleep(1)
        # 该链接通过抓包获得
        api_url = 'https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=' + str(page)
        print (api_url)
        rep = requests.get(url=api_url, headers=headers)
        for json in rep.json()['data']['statuses']:
            comment_ID = json['id']
            comments_ID.append(comment_ID)
        return comments_ID
lists = get_title_id()
print(lists)

NameError: name 'UserAgent' is not defined

In [1]:
# -*- coding: utf-8 -*-
import requests, random, re
import time
import os
import csv
import sys
import json
import importlib

importlib.reload(sys)
from fake_useragent import UserAgent
from lxml import etree

startTime = time.time()  # 记录起始时间
# 存为csv

path = os.getcwd() + "/weiboComments.csv"
csvfile = open(path, 'a', newline='', encoding='utf-8-sig')
writer = csv.writer(csvfile)
writer.writerow(('话题链接', '话题内容', '楼主ID', '楼主昵称', '楼主性别', '发布日期', '发布时间', '转发量', '评论量', '点赞量', '评论者ID', '评论者昵称', '评论者性别',
                 '评论日期', '评论时间', '评论内容'))  # csv头部

headers = {
    'Cookie': '_T_WM=22822641575; H5_wentry=H5; backURL=https%3A%2F%2Fm.weibo.cn%2F; ALF=1584226439; MLOGIN=1; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5RJaVYrb.BEuOvUQ8Ca2OO5JpX5K-hUgL.FoqESh-7eKzpShM2dJLoIp7LxKML1KBLBKnLxKqL1hnLBoMceoBfeh2EeKBN; SCF=AnRSOFp6QbWzfH1BqL4HB8my8eWNC5C33KhDq4Ko43RUIzs6rjJC49kIvz5_RcOJV2pVAQKvK2UbAd1Uh6j0pyo.; SUB=_2A25zQaQBDeRhGeBM71cR8SzNzzuIHXVQzcxJrDV6PUJbktAKLXD-kW1NRPYJXhsrLRnku_WvhsXi81eY0FM2oTtt; SUHB=0mxU9Kb_Ce6s6S; SSOLoginState=1581634641; WEIBOCN_FROM=1110106030; XSRF-TOKEN=dc7c27; M_WEIBOCN_PARAMS=oid%3D4471980021481431%26luicode%3D20000061%26lfid%3D4471980021481431%26uicode%3D20000061%26fid%3D4471980021481431',
    'Referer': 'https://m.weibo.cn/detail/4312409864846621',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.100 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}
comments_ID = []


def get_title_id():
    '''爬取战疫情首页的每个主题的ID'''
    for page in range(1, 21):  # 每个页面大约有18个话题
        headers = {
            "User-Agent": UserAgent().chrome  # chrome浏览器随机代理
        }
        time.sleep(1)
        # 该链接通过抓包获得
        api_url = 'https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=' + str(page)
        print(api_url)
        rep = requests.get(url=api_url, headers=headers)
        for json in rep.json()['data']['statuses']:
            comment_ID = json['id']
            comments_ID.append(comment_ID)


def spider_title(comment_ID):
    try:
        article_url = 'https://m.weibo.cn/detail/' + comment_ID
        print("article_url = ", article_url)
        html_text = requests.get(url=article_url, headers=headers).text
        # 话题内容
        find_title = re.findall('.*?"text": "(.*?)",.*?', html_text)[0]
        title_text = re.sub('<(S*?)[^>]*>.*?|<.*? />', '', find_title)  # 正则匹配掉html标签
        print("title_text = ", title_text)
        # 楼主ID
        title_user_id = re.findall('.*?"id": (.*?),.*?', html_text)[1]
        print("title_user_id = ", title_user_id)
        # 楼主昵称
        title_user_NicName = re.findall('.*?"screen_name": "(.*?)",.*?', html_text)[0]
        print("title_user_NicName = ", title_user_NicName)
        # 楼主性别
        title_user_gender = re.findall('.*?"gender": "(.*?)",.*?', html_text)[0]
        print("title_user_gender = ", title_user_gender)
        # 发布时间
        created_title_time = re.findall('.*?"created_at": "(.*?)".*?', html_text)[0].split(' ')
        # 日期
        if 'Feb' in created_title_time:
            title_created_YMD = "{}/{}/{}".format(created_title_time[-1], '02', created_title_time[2])
        elif 'Jan' in created_title_time:
            title_created_YMD = "{}/{}/{}".format(created_title_time[-1], '01', created_title_time[2])
        else:
            print('该时间不在疫情范围内，估计数据有误！URL = ')
            pass
        print("title_created_YMD = ", title_created_YMD)
        # 发布时间
        add_title_time = created_title_time[3]
        print("add_title_time = ", add_title_time)
        # 转发量
        reposts_count = re.findall('.*?"reposts_count": (.*?),.*?', html_text)[0]
        print("reposts_count = ", reposts_count)
        # 评论量
        comments_count = re.findall('.*?"comments_count": (.*?),.*?', html_text)[0]
        print("comments_count = ", comments_count)
        # 点赞量
        attitudes_count = re.findall('.*?"attitudes_count": (.*?),.*?', html_text)[0]
        print("attitudes_count = ", attitudes_count)
        comment_count = int(int(comments_count) / 20)  # 每个ajax一次加载20条数据，
        position1 = (article_url, title_text, title_user_id, title_user_NicName, title_user_gender, title_created_YMD,
                     add_title_time, reposts_count, comments_count, attitudes_count, " ", " ", " ", " ", " ", " ")
        writer.writerow((position1))  # 写入数据
        return comment_count
    except:
        pass


def get_page(comment_ID, max_id, id_type):
    params = {
        'max_id': max_id,
        'max_id_type': id_type
    }
    url = ' https://m.weibo.cn/comments/hotflow?id={}&mid={}&max_id'.format(comment_ID, comment_ID)
    try:
        r = requests.get(url, params=params, headers=headers)
        if r.status_code == 200:
            return r.json()
    except requests.ConnectionError as e:
        print('error', e.args)
        pass


def parse_page(jsondata):
    if jsondata:
        items = jsondata.get('data')
        item_max_id = {}
        item_max_id['max_id'] = items['max_id']
        item_max_id['max_id_type'] = items['max_id_type']
        return item_max_id


def write_csv(jsondata):
    for json in jsondata['data']['data']:
        # 用户ID
        user_id = json['user']['id']
        # 用户昵称
        user_name = json['user']['screen_name']
        # 用户性别,m表示男性，表示女性
        user_gender = json['user']['gender']
        # 获取评论
        comments_text = json['text']
        comment_text = re.sub('<(S*?)[^>]*>.*?|<.*? />', '', comments_text)  # 正则匹配掉html标签
        # 评论时间
        created_times = json['created_at'].split(' ')
        if 'Feb' in created_times:
            created_YMD = "{}/{}/{}".format(created_times[-1], '02', created_times[2])
        elif 'Jan' in created_times:
            created_YMD = "{}/{}/{}".format(created_times[-1], '01', created_times[2])
        else:
            print('该时间不在疫情范围内，估计数据有误！')
            pass
        created_time = created_times[3]  # 评论时间时分秒
        #         if len(comment_text) != 0:
        position2 = (
        " ", " ", " ", " ", " ", " ", " ", " ", " ", " ", user_id, user_name, user_gender, created_YMD, created_time,
        comment_text)
        writer.writerow((position2))  # 写入数据


#         print (user_id, user_name, user_gender, created_YMD, created_time)
def main():
    count_title = len(comments_ID)
    for count, comment_ID in enumerate(comments_ID):
        print("正在爬取第%s个话题，一共找到个%s话题需要爬取" % (count + 1, count_title))
        maxPage = spider_title(comment_ID)  # 评论占的最大页数
        print('maxPage = ', maxPage)
        m_id = 0
        id_type = 0
        if maxPage != 0:  # 小于20条评论的不需要循环
            try:
                for page in range(0, maxPage):  # 用评论数量控制循环
                    jsondata = get_page(comment_ID, m_id, id_type)
                    write_csv(jsondata)
                    results = parse_page(jsondata)
                    time.sleep(1)
                    m_id = results['max_id']
                    id_type = results['max_id_type']
            except:
                pass
        print("--------------------------分隔符---------------------------")
    csvfile.close()


if __name__ == '__main__':
    get_title_id()
    main()
    endTime = time.time()  # 获取结束时的时间
    useTime = (endTime - startTime) / 60
    print("该次所获的信息一共使用%s分钟" % useTime)

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "S:\Anaconda\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "S:\Anaconda\lib\http\client.py", line 1244, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "S:\Anaconda\lib\http\client.py", line 1290, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "S:\Anaconda\lib\http\client.py", line 1239, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "S:\Anaconda\lib\http\client.py", line 1026, in _send_output
    self.send(msg)
  File "S:\Anaconda\lib\http\client.py", line 966, in send
    self.connect()
  File "S:\Anaconda\lib\http\client.py", line 938, in connect
    (self.host,self.port), self.timeout, self.source_address)
  File "S:\Anaconda\lib\socket.py", line 727, in 

https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=1
https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=2
https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=3
https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=4
https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=5
https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=6
https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=7
https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=8
https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=9
https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=10
https://m.weibo.cn/api/feed/trendtop?containerid=102803_ctg1_600059_-_ctg1_600059&page=11
https://m.weibo.cn/

KeyboardInterrupt: 

In [1]:
from html.parser import HTMLParser
import re


def filter_emoji(desstr, restr=''):
    # 过滤表情
    try:
        co = re.compile(u'[\U00010000-\U0010ffff]')
    except re.error:
        co = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
    return co.sub(restr, desstr)


def data_filter(content, lower):
    # 1.去掉html标记
    html = content.strip()
    html = html.strip("\n")
    result = []
    parse = HTMLParser()
    parse.handle_data = result.append
    parse.feed(html)
    parse.close()
    content = "".join(result)

    # 2.去掉url标记
    r1 = '''http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'''
    content = re.sub(r1, '', content)

    # 3.去掉@标记与@某人的标记
    r2 = "(@.*?)+ "
    content = re.sub(r2, '', content)
    content = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", " ", content)

    # 4.去掉一些特殊字符
    r3 = u'[’!"#$%&\'()*+,-./:;<=>?@；；：．｜～\≧▽—°❄×☝➡️❤️⭐■⏰🎂🌕👇🤔📷🎤🉑➕🥑👍📒🥚📹🆔🍊☁✨📍💝👗🚄🍀🐾🍓🐋▲🔺🔗❗🦐🐫🧜‍🏃👇➕♥♀☀●巜「」☕／↓→<=>?@⁄•ω★💊🙈☕💰😂·、…★、…【】《》『』（）？“”‘’！[\\]^_`{|}~]+'
    content = re.sub(r3, '', content)

    # 5.过滤表情
    content = filter_emoji(content, restr='')
    content = re.sub(r"\[\S+\]", "", content)  # 去除表情符号

    # 6.合并正文中过多的空格
    content = re.sub(r"\s+", " ", content)

    # 7.过滤掉\ax0
    content = content.strip().replace(u'\u3000', u'').replace(u'\xa0', u'')

    # 8.是否转换为小写
    if lower:
        content = str(content).lower()

    return content.replace(" ", "")

In [4]:
import pandas

df = pandas.read_csv('./test.csv')
df
for i in range(5592):
    df.loc[i,'text'] = data_filter(df.loc[i,'text'],True)

df.to_csv('new2.csv',index=False, encoding='utf-8')

In [5]:
d = pandas.read_csv('./new2.csv')
d.dropna(subset=['text'],inplace=True)
d.to_csv('new_null2.csv',index=False,encoding='utf-8')

In [6]:
dd = pandas.read_csv('./new_null2.csv')
dd

,wid,time,text,uid,like_count,username,following,followed,gender
0,4528782646188528,Mon Jul 20 15:16:26 +0800 2020,旦总，什么平台可以购买,3053040100,0,FF55YY,26,48,f
1,4528783585968776,Mon Jul 20 15:20:09 +0800 2020,你去中国邮政网站上看一下,1679583062,数据缺失,lovelife旦旦,581,956992,m
2,4528836165505646,Mon Jul 20 18:49:06 +0800 2020,一定要注意安全啊,5694596438,0,乖XY喵困,225,426,f
3,4529198720886701,Tue Jul 21 18:49:46 +0800 2020,央企之耻际华集团赶快退市，别骗了,1940219091,0,专业牛逼,89,117,m
4,4529149753170364,Tue Jul 21 15:35:11 +0800 2020,有个毛用啊，看看际华集团的业绩，过去几年扣非亏损上百亿了，这样的央企就知道作秀有什么用呢，坑...,1940219091,0,专业牛逼,89,117,m
...,...,...,...,...,...,...,...,...,...
5279,4529406141270711,Wed Jul 22 08:33:58 +0800 2020,哈哈哈哈,1893156807,0,夏洛特的网的微博,3432,537,f
5280,4529446608702076,Wed Jul 22 11:14:46 +0800 2020,最新澄清，说香港能够处理疫情，一是大陆水灾等严重，依然忙碌，不想打扰二是大陆医护人员没有香港...,1826838442,0,云界漫步,75,23598,m
5281,4529396649564473,Wed Jul 22 07:56:16 +0800 2020,你仔细品,6535520087,0,老男人之终极腊肉,32,1,m
5282,4529394641797533,Wed Jul 22 07:48:16 +0800 2020,又他妈倒贴他们自己想办法去,5764887846,0,pawn丶Lambl1ngCH,34,34,m


In [56]:
for i in range(12529):
    print(df.loc[i,'text'])

1死亡率看上去在下降是因为感染人数增加太快，其实绝对数比北欧其他三国死亡总和还多2更重要的是，其后期修改了死亡定义，只要死者有其他病症的，都不按新冠算。
请问有哪些指定或认可的机构呢
现在看看公布的数据都太可怕了，伊朗说有2500万感染，印度说孟买有20的人口感染，鹅毛最狠，说莫斯科有60有抗体了美国感染人数放大十倍真的是小儿科了，都不知道信谁
正好相反，说明新冠病毒不可怕
这不就说明死亡率缩小成110么。。。如果美国有超过3000万感染，平均起来抗体率10了，具体到人口稠密的地方2030，那应该快跟ny一样进入下行通道了
全世界这个感染数字都被低估，美国测试数量还是挺大的
按武汉当时的状态，至少病毒到了冬天很可怕。
她对得起胸前的十字架吗
美女主持，可惜太能编谎言
慰安妇
生肉
发言人的意思就是，trump测试很多次了，都没事，所以他不用戴口罩，换句话来说，trump有测试的特权啦所以他不用戴口罩赤裸裸炫耀吗
大概意思是关于抗议者过激行为都有法可依，会得到处罚，执政部门出动包括fbi在内的所有力量进行控制，南西佩洛西颂赞的美丽风景线社及违法该抓不误并会因砸毁雕像，用雷射光来照射施法人员涉及违法，总而言之一句话，在香港的暴徒对付警察的所有手段都是美丽风景线，到了美国就违法明目张胆的双标
听不懂
我的
撒谎精
说的啥
人造美女
每天起床第一句，先给老美打个气新增不够七万例，都要说声对不起川普川普告诉我，你的底线在哪里拜拜，测试剂，有钱才能用得起，生病住院七万起，平民平民不可以拜拜，隔离衣，医用口罩消毒剂，防护面罩呼吸机，一样都没准备齐来来，别着急，勒索不会只一起，日本韩国意大利，快给爸爸交供给
博主还未休息，好辛苦
转发微博
欧美宣传功夫一流，先画个大饼，至于能不能做到以后谁还关心记得几年前欧洲债务危机，援助希腊几百亿欧元都拿不出来。这还都是什么发行债券，什么基金，纯属吹牛画大饼
外国人真有钱啊
舍得戴口罩了
我感想可发表，我只是激动坐上了沙发。
经济复苏的关键在于有效控制疫情
转发微博
放水了。。。要跌一跌
口罩
路过
抱团取暖

早啊
早
哈哈
没有用的。
转发微博
疫情没结束，会浪费了
炒房阿
每天起床第一句，先给老美打个气新增不够七万例，都要说声对不起川普川普告诉我，你的底线在哪里拜拜，测试剂，有钱才能用得起，生病住院七万起，平民平民不可以拜拜，隔离衣，医用口

现在对海对面，抱着看笑话的心态就行，那边现在是魔幻的时代。
选了个满嘴谎话跑火车的人当总统，美国人民全民被pua加失智
他是说新冠确诊吧
这样的话，武汉市检测量排世界第三。
5千万里就有400万得新冠，不知他的自信是从哪里来的
中国光武汉一个市检验的就不下千万
查一个阳性，查一个阳性，要说美国这核酸检测太有成就感了。
没意见让你得瑟
控制得好不用检测，不是更好吗。建国同志的逻辑像个小孩子。
几个月前检测的不能保证现在没有被感染
广东检测了多少
印度也是要玩儿完
常规操作直接忽视中国和俄罗斯检测数量
图二是真的吗
记者不吹牛皮会死特不吹真的会死。
胸前的红领巾已红里透紫了
这是个傻子，一无所知。
北京我一天可以检测50w人以上，累积检测超1100w我排名第三
我们属于外星球
你们牛，行了吧
cia是不是每天给川普看特刊
蜜汁逻辑
中国在他眼里不是个国家了一个国家总统这么胡说八道，到底是好还是不好
武汉排名第三
艹。老子武汉10天就查了900万人，你5000多吗
美国技术就是好，检出概率就是高，以后百分之百。有关部门给建国涨工资
美国事事要争第一，就连得病也是如此。
5000万好像只有中国某个省一半人口
阿三很快就要第二了
哈哈笑死人了老特真白痴
检测结果2周才能拿到完全失去了测试后立刻追踪的意义
cctv9应该拍纪录片检测过的人都出下镜
他就是个娱乐大咖
厉害了，宇宙第一
放弃吧
中国是神，中国人是神人，神人的境界只有上帝懂
我x北京排第三了已经超1100万了
迅雷不及掩耳盗铃响叮当
被开除球藉了
故意忽略我们数据
中小学生如何安全过暑假这42条请牢记
教育分享啦中小学生如何安全过暑假这42条请牢记
中小学生如何安全过暑假这42条请牢记
浙大的强奸犯不调查一下么浙大的强奸犯不调查一下么就因为他是少数民族因为他是贫困地区不贫困的毕业生
浙大很厉害的浙大18年还有学生在宾馆吸毒被留校察看，18年有人在图书馆猥亵女学生留校察看，19年还有学生在寝室吸毒被留校察看，20年有人强奸被留校察看不得不说，这大学真tm牛逼
转发微博
大连大学研究生毕业证
有什么用，逃过这些，也逃不过强奸，猥亵，吸毒，浙大的事你们教育部难逃罪责
高校解封
是的，宝贝
冲绳蛮惨的某国真的是害己害人啊
受压迫的冲绳人民站起来
殖民地的悲哀
没办法，主子得供着
没有主权的国，瞎叫唤。毛爷爷抗美援朝才有今天中

这热闹成自己的了
自己的车停哪儿不知道长什么样儿不知道
这瓜吃的有点疼
想问下这个保险能赔吗

这不是我高中英语阅读的故事吗
有天早上我爸说好像有个卡车撞了院墙把停那的车给砸了然后兴冲冲的跑去看热闹。然后过了一会儿回来跟我们说。哎呀是我的车被砸了哈哈哈哈
吃瓜吃到自己头上
这两天看到一句话特别好，大致意思是疫情后，中国面对漏洞迅速行动，实事求是地反思，改革。美国死了140000人了，至今听到任何反思的声音吗更别提从国家层面改革。
精准防控不伤全局，将是中国抗疫常态。加油
新冠让网络时代腾飞，都不敢怎么出门了，以后就是云时代了
能提出模拟时代终结，说明这个人太老旧了
这不就是大数据嘛，我们都用这个两年了，老外才反应过来
而不是，才是是不是打错了
意思就是大数据时代，你隐私在我眼里看的一清二楚呗
下个时代是电子时代
不是早就是大数据时代了么
凡叛离故土者，心中最大的期待莫过于故土衰弱，自己发达，可以居高临下地鄙视穷亲戚，某日衣锦还乡受到尊崇追捧。然而事与愿违，自己发展越发窘困，故人却风生水起名震天下，那种心理落差便是憎恨之源。只要能踩死对方，怎样都愿意。
中国人早就开始干的事，洋鬼子才明白啊
说实在的，本来是通过一场战争来决定胜负的，修昔底德式的陷阱，现在新冠用一把尺子把大家都比了一下，不用打仗了。
吃相难看
背后是利益之争。
到底是不是名吃，是不是好吃，都是民众自己吃完评吧，这个地方协会，那个省协会的，能代表本地人民群众的口味吗
用胃投票
不为钱不为名难道为良心么。但凡稍微上点档次的饭店宾馆，最基本的一个蔬菜都是几十起步的
有些事情真的不能不了了之

王多余呢
有真的开门了的影院嘛
中国医护太给力，逆天保命第一例
中国医生，真棒
中国医生，真棒
全球首位
医护人员辛苦肺移植要多少钱啊
医护人员很棒，辛苦啦大爷要健健康康的
辛苦了辛苦了致敬
太幸运了，老爷子加油
加油，给中国医生点赞。
大难不死必有后福
那个女医生很激动啊主刀大夫
为中国医护工作者打call，你们太优秀致敬
都辛苦了
我的天啊太厉害了吧中国医护
李文亮那么年轻为啥没躲过去
医疗水平进步相当大啊
脸怎么这么黑，会不会还没有度过危险期


全球首位，太棒了，中国医生
大家都很棒果然意志坚强不能轻易放弃
大难不死必有后福，以后平安
必有后福

加油
为中国医生点赞加油
中国医生，真棒
好棒

他吃啥了
还需要

湖北地级市，五月底初中复学检测费180元
免费的背后，是财政支出。hk政府没有那么多财权，而大陆不一样。你觉得呢
不知道汉人是什么地位了
不是每人刚发了一万吗
我们免费是因为不免费很多人省一点钱都不愿意去检测，香港人不至于两千都没有
评论区什么情况
看到有人发出的视频普通检测1050hkd，48小时出结果，加急另付600hkd

新疆富裕
还是人少北京那么长时间也只是有限部分免费检测
乌鲁木齐从哪传染的到现在都没有通报
哈哈，看评论
因为香港是资本主义，新建是社会主义
北京也没全部啊
都是定数
越南，台湾，非洲都低
没有疫苗人类社会真的无法恢复正常运转。
天热病毒不好存活
疟疾区
普吉岛的鱼虾蟹们
防疫做的好的，都祝福他们，替他们开心

谋杀儿童罪
他们既不关注老人也不重视孩子，中年固执失智，青年头铁作死，真是
友谊是阳光，让心灵共同成长友谊是春雨，让微笑共同绽放友谊是翅膀，让幸福并肩飞翔友谊是风帆，让美好一路远航。祝朋友幸福快乐永远
早

愿往后余生，冷暖有相知，喜乐有分享
早上好

有论文链接吗
请教一下，国内之前说为3，重症率为6左右现在重症率降到1，死亡率也会下降吗
早干嘛去了。
最终还是戴起来
第一名的作业给都不抄还骂第一名还联合班里的垃圾学生一起搞第一名
别戴呀，皿煮兹有呢
几号回去我10月底
你确定航班了吗
不错不错，希望不要再改签了
早上好
人能相遇，已是不易心若相知，更要珍惜。
澳洲的方法看看就好。
如果您方便的话，恳请您帮帮忙转发我的水滴筹，713我爸被家里煤气罐爆炸严重烧伤烧伤面积高达90％现在已经负债累累好多爱心人士给我爸献血的捐款的医生说治疗和植皮费用加起来保守治疗至少要100万对于我们土里刨食的农村人真的是天文数字我又不想失去我爸我想要一个完整的家有爸妈都在的家

每天坚持互动，感谢回访
人能相遇，已是不易心若相知，更要珍惜。

不会想送到深圳来吧
一扇心里有美景的窗

风景真好

风景太美了。

早上好啊，来看看美丽的意大利
早
这里是
意大利
哪里
意大利
我觉得吧，今年活着就好
没想到居然一半一半，青青我建议你还是在家里上网课，因为我不知道荷兰疫情的情况，虽然上网课效率是真的没那么高我也经历了半年了，但是我觉得还是安全健康第一。如果最后选择了部分到学校上课，青青一定要做好防护措施呀
在家上课命比较重要
安全最重要
我觉得青青很自律啦

心酸

爬窗太危险，但此时我只想说，他做的对第一幅图片里充满了浓郁的忧伤，让我在这场让人束手无策的病毒抗战里，感知到爱与温情。
难过
这种无力感，愿不好的一切，快点过去
我还记得，武汉那个追着灵车，至死都见不到母亲一面的女孩，深夜追车，一声声的妈很难受
真好哭，
今年过年的时候武汉有个女生，眼看着自己父母感染新冠又得不到有效救治而离世，自己也感染了病毒，那种跟死神对视的绝望真的很悲凉。
看哭了，人生最残忍的事，我是那么的爱你，看你死去，我却无能为力
看着太难受了
又是被感动到地一天
爱不分国界但分人性
愿上帝保佑你


我眼泪不值钱
将来的历史会揭露造成这次疫情大范围蔓延的真相，并谴责在这次疫情里为了各自的利益去隐瞒欺上瞒下推罪污蔑的各种罪人，并写进历史教科书警示后人但是遗憾的是我们这代人因为在某种环境下是看不到的，留给儿孙们看吧
武汉当时好多人也经历过没见到家人最后一面的情况唉太绝望了
世界上每一个孩子都深情的爱着自己的母亲
唉，新冠肺炎带给全世界这么大的悲痛。希望那些首脑人物能认真点，像中国一样重视疫情，全世界携手抗击疫情。这样才能尽快结束疫情。
离家的506天，想家，无与伦比的想家
愿疫情早早结束
这得多伤心啊眼睁睁看着自己最亲的人死去

看了好想哭
唉
最看不了这种是个孝顺的好孩子

中国要是这样干就是给社会添乱
唉
感动，妈妈是世界上最重要的人，但是对妈妈来说，孩子才是自己最爱最牵挂的。
当时的武汉不就是这样吗，满城绝望

不能看这种新闻，眼泪汪汪，

想到了血族里面的巴西小哥
但愿天堂没有病痛
不幸的是无法挽救母亲的生命，幸运的是母亲最爱的孩子陪她走完最后一程

给我哭
唉
他妈妈一直在等着看他最后一眼

美国在全球的生化研究所应该用导弹轰平
愿全世界的人都能平安健康愿疫情早日结束
爱别离，真是人生最苦。
我在评论区找有没有人说巴铁的，居然没找到
突然联想到老爸为什么很依恋老妈，分房睡坚持不了几天就爬过去找她，可能因为他在八岁就失去了母亲，反思自己很少考虑爸妈内心的脆弱
最纯，最真的感情百善孝为先活着的人还是要乐观面对。加油
泪目
感动
真的感谢我们祖国妈妈不抛弃不放弃，万众一心，倾尽全力抗过这次疫情
造成这一切的罪魁祸首是
啊难受啊

不久前刚经历过最亲的人在icu却无法探视，每天只能在icu地上坐着，等着医生护士的电话疫情期间，不得探视还好，一个多月

In [41]:
# np.where(condition, x, y)
# 满足条件(condition)，输出x，不满足输出y。
import time
import pandas as pd
import numpy as np
def change_p(path):
    data = pd.read_csv(path)
    print(data.loc['text'])
    print("-------------文件已经读入内存----------")
    print("-------------数据处理完毕,开始写入----------")
    data.to_csv('new.csv',index=False, encoding='utf-8')
if __name__ == '__main__':
    start=time.time()
    path="nCov_12k_1.csv"
    change_p(path)
    end=time.time()
    print("总共耗时",end-start)

0        1495100927
1        6236230137
2        1862832312
3        1762404844
4        2283499810
            ...    
12524    6292818873
12525    7382080692
12526    6509299536
12527    2369418011
12528    6068169191
Name: uid, Length: 12529, dtype: int64
-------------文件已经读入内存----------
-------------数据处理完毕,开始写入----------
总共耗时 0.5215234756469727


In [2]:
import pandas as pd

df = pd.read_csv("test_after.csv")
df

,wid,time,text,uid,like_count,username,following,followed,gender
0,4529878760164897,Thu Jul 23 15:52:00 +0800 2020,不需要这么做因为5g天线放进口罩已经足以控制你们了,3101630921,96,我姓倪别的不能说了,194,2219,m
1,4529878760164897,Thu Jul 23 15:52:00 +0800 2020,不需要这么做因为5g天线放进口罩已经足以控制你们了,3101630921,96,我姓倪别的不能说了,194,2219,m
2,4529882221249717,Thu Jul 23 16:05:44 +0800 2020,比亚迪的口罩有5g芯片，千万别用,2296948284,数据缺失,成成成小四,153,312,m
3,4529882136315032,Thu Jul 23 16:05:24 +0800 2020,5g天线我控制这些木鱼脑袋干啥呀，闲的吗,6458970173,数据缺失,不超过十个,14,17,m
4,4529881889905876,Thu Jul 23 16:04:26 +0800 2020,现在我每天在这里8买各种赛今天又收获满满cbanba典静其拍啥都有一边玩一边收米今天提线六千...,2173015011,233,月季哥哥阿,67,1553,m
...,...,...,...,...,...,...,...,...,...
6358,4529607707727470,Wed Jul 22 21:54:55 +0800 2020,不仅你们zf，还有你们国民，自大，不尊重科学,3729766527,0,JackCapirate,98,6,m
6359,4529601349422180,Wed Jul 22 21:29:40 +0800 2020,可悲,1934013343,0,Tonycha,463,1136,m
6360,4529787836042719,Thu Jul 23 09:50:42 +0800 2020,丢了人最宝贵的生命,5220633096,0,红堤葡萄,1156,1277,m
6361,4529591936361150,Wed Jul 22 20:52:16 +0800 2020,他们很忙，忙着选举,6001990422,0,安妮冯李,83,11,f


In [17]:
import time
import pandas as pd
import numpy as np

df = pd.read_csv("test_after.csv")
newdata = df.drop_duplicates(subset=['wid'],keep='first')
newdata

,wid,time,text,uid,like_count,username,following,followed,gender
0,4529878760164897,Thu Jul 23 15:52:00 +0800 2020,不需要这么做因为5g天线放进口罩已经足以控制你们了,3101630921,96,我姓倪别的不能说了,194,2219,m
2,4529882221249717,Thu Jul 23 16:05:44 +0800 2020,比亚迪的口罩有5g芯片，千万别用,2296948284,数据缺失,成成成小四,153,312,m
3,4529882136315032,Thu Jul 23 16:05:24 +0800 2020,5g天线我控制这些木鱼脑袋干啥呀，闲的吗,6458970173,数据缺失,不超过十个,14,17,m
4,4529881889905876,Thu Jul 23 16:04:26 +0800 2020,现在我每天在这里8买各种赛今天又收获满满cbanba典静其拍啥都有一边玩一边收米今天提线六千...,2173015011,233,月季哥哥阿,67,1553,m
5,4529878547050775,Thu Jul 23 15:51:09 +0800 2020,这还真的需要专门辟谣,1895755777,38,行者居无修,126,105,f
...,...,...,...,...,...,...,...,...,...
6335,4530058636558487,Fri Jul 24 03:46:45 +0800 2020,实在太搞笑啊,6681032486,0,覃承福_霸道,51,0,f
6347,4529911072821598,Thu Jul 23 18:00:23 +0800 2020,你也蛮有研究的,1866293720,数据缺失,普拉多的兔子,464,19114,f
6350,4529901954409178,Thu Jul 23 17:24:10 +0800 2020,研究的很深入,1866286420,数据缺失,没有昵称的是我吗,36,2579,f
6352,4529911001780961,Thu Jul 23 18:00:07 +0800 2020,卡路里,1866293720,数据缺失,普拉多的兔子,464,19114,f


In [16]:
df.to_csv("after_del_same.csv",index=False,encoding='utf-8')
df = pd.read_csv("after_del_same.csv")
df

,wid,time,text,uid,like_count,username,following,followed,gender
0,4529878760164897,Thu Jul 23 15:52:00 +0800 2020,不需要这么做因为5g天线放进口罩已经足以控制你们了,3101630921,96,我姓倪别的不能说了,194,2219,m
1,4529878760164897,Thu Jul 23 15:52:00 +0800 2020,不需要这么做因为5g天线放进口罩已经足以控制你们了,3101630921,96,我姓倪别的不能说了,194,2219,m
2,4529882221249717,Thu Jul 23 16:05:44 +0800 2020,比亚迪的口罩有5g芯片，千万别用,2296948284,数据缺失,成成成小四,153,312,m
3,4529882136315032,Thu Jul 23 16:05:24 +0800 2020,5g天线我控制这些木鱼脑袋干啥呀，闲的吗,6458970173,数据缺失,不超过十个,14,17,m
4,4529881889905876,Thu Jul 23 16:04:26 +0800 2020,现在我每天在这里8买各种赛今天又收获满满cbanba典静其拍啥都有一边玩一边收米今天提线六千...,2173015011,233,月季哥哥阿,67,1553,m
...,...,...,...,...,...,...,...,...,...
6358,4529607707727470,Wed Jul 22 21:54:55 +0800 2020,不仅你们zf，还有你们国民，自大，不尊重科学,3729766527,0,JackCapirate,98,6,m
6359,4529601349422180,Wed Jul 22 21:29:40 +0800 2020,可悲,1934013343,0,Tonycha,463,1136,m
6360,4529787836042719,Thu Jul 23 09:50:42 +0800 2020,丢了人最宝贵的生命,5220633096,0,红堤葡萄,1156,1277,m
6361,4529591936361150,Wed Jul 22 20:52:16 +0800 2020,他们很忙，忙着选举,6001990422,0,安妮冯李,83,11,f
